## Student report

- df_passed_ex_enhance -> counts_df  
- dfs['skill_building_line'] -> course_df
- dfs['students'] -> students_df
- df_vehicle_pc_avg -> slalom_avg
- df_passed_ex -> passed_ex_df

In [1]:
header_center_text='header_center_text foo'
footer_center_text='footer_center_text bar'
Header_caption=((header_center_text))

In [97]:
# Group Variables
gasnor = int((df_passed_ex_enhance['count']['Slalom'].agg('mean'))) # group_average_slalom_runs
gaspoce = int((df_passed_ex_enhance['Slalom Passed'].mean())*100) # group_average_slalom_prcnt_completed
gasaoep = int(((df_passed_ex_enhance['av_score']['Slalom'].mean())*100)) # group_average_slalom_ex_prcnt
gasaovc = int((dfs['skill_building_line'].loc[dfs['skill_building_line']['exercise'] == 'Slalom']['%_of_vehicle'].mean())*100) # group_average_slalom_vehicle_control
galnor = int((df_passed_ex_enhance['count']['Lane Change'].agg('mean'))) #.astype(int) # group_average_lnch_runs
galpoce = int((df_passed_ex_enhance['LnCh Passed'].mean())*100) # group_average_lnch_prcnt_completed
galaoep = int(((df_passed_ex_enhance['av_score']['Lane Change'].mean())*100)) # group_average_lnch_ex_prcnt
galaovc = int((dfs['skill_building_line'].loc[dfs['skill_building_line']['exercise'] == 'Lane Change']['%_of_vehicle'].mean())*100) # group_average_lnch_vehicle_control
mseg_t_pre = str(df_final_exercise['f_time'].mean())
mseg_t = mseg_t_pre[mseg_t_pre.find(':')+1 : mseg_t_pre.find('.')+3 : ] # mse_group_av_time
mseg_c = int((df_final_exercise['cones'].mean())) # mse_group_av_cones
mseg_g = int((df_final_exercise['gates'].mean())) # mse_group_av_gates
mseg_perf = int((df_final_exercise['final_result'].mean())*100) # mse_group_av_performance
mseg_per = int((df_final_exercise['final_result'].quantile())*100) # mse_group_av_percentile
mse_obj = str(pd.to_timedelta(dfs['final_exercise'].loc[0, 'ideal_time sec'], unit='s'))[10:] # mse_objective_obj
mseg_rev_pre = str(df_final_exercise['rev_slalom'].mean()) # strings mse_group_used_in_rev
"""New Change in slicing"""
mseg_rev = re.findall('days 00:00:(.*\..{2}|.*$)', mseg_rev_pre)[0] # strings mse_group_used_in_rev <- Chnaged May 2021
mseg_rev_pc = int(round(((df_final_exercise['rev_%'].mean())*100), 0))

In [247]:
# Temporary Variables waiting for code
msed_per = np.nan # MSE Driver Percentile (Quantile)
msed_rev = np.nan # MSE Driver % Used in Reverse
msed_rev_time = np.nan # MSE Driver Reverse Time
paragraph = np.nan # Lead Instructor Feedback

In [124]:
country = dfs['course_generals'].loc[0, 'country']
make = dfs['car_information'].loc[0, 'make']
program = dfs['course_generals'].loc[0, 'program']
course_date = dfs['course_generals'].loc[0, 'date']
latacc = dfs['car_information'].loc[0, 'latacc']

#Final report DataFrame Lineup
df_report = pd.DataFrame(dfs['students'], columns=['fullname', 'studentId'])
df_report['company'] = np.nan
df_report['program'] = program
df_report['date'] = course_date

# Versiones en ambos idiomas
if country == 'MX':
    df_report['vehicle'] = (make + ' (Capacidad ' + latacc.astype(str) + 'g)')
else:
    df_report['vehicle'] = (make + ' (' + latacc.astype(str) + 'g Capability)')

# Slalom variables
df_report['s_no_runs'] = pd.merge(left=df_report, right=(df_passed_ex_enhance['count']['Slalom']).astype(int), left_on='fullname', right_index=True)['Slalom']
df_report['s_passed'] = pd.merge(left=df_report, right=df_passed_ex_enhance['passed']['Slalom'], left_on='fullname', right_index=True)['Slalom']
df_report['prcnt_s_pass'] = pd.merge(left=df_report, right=round((df_passed_ex_enhance['Slalom Passed'])*100, 0), left_on='fullname', right_index=True)['Slalom Passed']
df_report['avg_ex_control_s'] = pd.merge(left=df_report, right=round((df_passed_ex_enhance['av_score']['Slalom'])*100, 0), left_on='fullname', right_index=True)['Slalom']
df_report['avg_v_control_s'] = pd.merge(left=df_report, right=round((df_vehicle_pc_avg['vehicle_pc_avg']['Slalom'])*100, 0), left_on='fullname', right_index=True)['Slalom']
df_report['slalom_max'] = pd.merge(left=df_report, right=round((df_passed_ex['end_score']['Slalom'])*100), left_on='fullname', right_index=True)['Slalom']

#LnCh Variables
df_report['lc_no_runs'] = pd.merge(left=df_report, right=(df_passed_ex_enhance['count']['Lane Change']).astype(int), left_on='fullname', right_index=True)['Lane Change']
df_report['lc_passed'] = pd.merge(left=df_report, right=df_passed_ex_enhance['passed']['Lane Change'], left_on='fullname', right_index=True)['Lane Change']
df_report['prcnt_lc_pass'] = pd.merge(left=df_report, right=round((df_passed_ex_enhance['LnCh Passed'])*100, 0), left_on='fullname', right_index=True)['LnCh Passed']
df_report['avg_ex_control_lc'] = pd.merge(left=df_report, right=round((df_passed_ex_enhance['av_score']['Lane Change'])*100, 0), left_on='fullname', right_index=True)['Lane Change']
df_report['avg_v_control_lc'] = pd.merge(left=df_report, right=round((df_vehicle_pc_avg['vehicle_pc_avg']['Lane Change'])*100, 0), left_on='fullname', right_index=True)['Lane Change']
df_report['lnch_max'] = pd.merge(left=df_report, right=round((df_passed_ex['end_score']['Lane Change'])*100), left_on='fullname', right_index=True)['Lane Change']

In [136]:
# Setup the right company
def set_right_company_student_report(row, student_company_map, client_name):
    """
    Set's the company for a student in the student report. If the company is not known for a student then put client name
    """
    company = client_name
    if row["studentId"] in student_company_map:
        company = student_company_map[row["studentId"]]
    return company


student_company_map = dfs['students'].set_index("studentId")["company"].to_dict()
client = dfs['course_generals'].loc[0, 'client']
df_report["company"] = df_report.apply(lambda row: set_right_company_student_report(row, student_company_map, client), axis=1)

In [145]:
# Including the right comments
student_comment_map = dfs["instructor_comments"].set_index("studentId")["comment"].to_dict()
df_report["comments"] = df_report["studentId"].map(student_comment_map)

In [147]:
df_final_exercise.loc[df_final_exercise['final_result'] < .8]

,exercise,studentId,participant,car,stress,rev_slalom,rev_%,slalom,LnCh,cones,gates,f_time,final_result
2,Final Exercise,RSH19681019,Héctor Ruiz Flores,1,0,0 days 00:00:19.450000,0.24,52,43,1,0,0 days 00:01:24.030000,0.78
3,Final Exercise,CZJ19641114,Jesús Cruz González,1,0,0 days 00:00:18.090000,0.22,70,60,4,0,0 days 00:01:29.420000,0.70
5,Final Exercise,VOG19770307,German Enoc Velazco Gallardo,1,0,0 days 00:00:17.950000,0.23,74,82,3,0,0 days 00:01:25.170000,0.77
6,Final Exercise,LZL19630210,Luis Martín Lima González,1,0,0 days 00:00:22.190000,0.25,48,59,3,1,0 days 00:01:40.140000,0.55
9,Final Exercise,LZL19630210,Luis Martín Lima González,1,1,0 days 00:00:17.400000,0.22,62,66,3,1,0 days 00:01:28.810000,0.71


In [228]:
# Important change in how dates are read (Cambio Importante en como se leen las fechas)
# Final Exercise Variables - Possible setting as table for multiple occurances
mse_report = pd.DataFrame(df_final_exercise.replace(np.nan, '-')
                          .drop(['exercise', 'rev_slalom'], axis=1)
                        )

mse_report['f_time'] = mse_report['f_time'].astype(str).str.extract('days 00:(.*\..{2}|.*$)')

if country == 'MX':
    mse_report['stress'].replace((1, 0), ('Alto', 'Bajo'), inplace=True)
else:
    mse_report['stress'].replace((1, 0), ('High', 'Low'), inplace=True)

mse_report[['rev_%', 'final_result']] = mse_report[['rev_%', 'final_result']].apply(lambda x: x*100).astype(int)
mse_report.rename(columns={'rev_%' : 'rev_pc'}, inplace=True)

for i, r in mse_report.iterrows():
    if '.' in r.f_time:
        pass
    else:
        r = r.copy()
        ov = r.f_time
        ov = ov+".01"
        mse_report.loc[i, 'f_time'] = ov

mse_report = mse_report.groupby('participant').apply(lambda x: x.to_dict(orient='records'))

### Jinga variable reads

In [181]:
jinga_dir = "../../../static/reports"
tmplt_data = pd.read_excel(f'{jinga_dir}/jinja_variables.xlsx', skiprows=2)
tmplt_context = dict(zip(tmplt_data['var'], tmplt_data['value']))

In [185]:
'-------------------------------------------------------------'
'Nex Table For Final Exercise'
'-------------------------------------------------------------'

student = "Héctor Ruiz Flores"
mse_student = df_final_exercise[df_final_exercise['studentId'] == studentId].index.tolist()
# finalx_df_slice = finalx_df.iloc[7]
mse_data = pd.DataFrame(df_final_exercise.iloc[mse_student], columns=['stress', 'f_time', 'rev_%', 'slalom', 'LnCh', 'cones', 'gates', 'final_result'])
mse_data[['rev_%', 'final_result']] = (mse_data[['rev_%', 'final_result']]*100).astype(int)
mse_data['f_time'] = mse_data['f_time'].astype(str).str.extract('days 00:(.*\..{2}|.*$)')
if country == 'MX':
    mse_data['stress'].replace((1, 0), ('Alto', 'Bajo'), inplace=True)
else:
    mse_data['stress'].replace((1, 0), ('High', 'Low'), inplace=True)
mse_data['rev_%'] = mse_data['rev_%'].astype(str) + '%'
mse_data['slalom'] = mse_data['slalom'].astype(str) + '%'
mse_data['LnCh'] = mse_data['LnCh'].astype(str) + '%'
mse_data['final_result'] = mse_data['final_result'].astype(str) + '%'

In [186]:
mse_data

,stress,f_time,rev_%,slalom,LnCh,cones,gates,final_result
2,Bajo,01:24.03,24%,52%,43%,1,0,78%
10,Alto,01:13.43,20%,59%,63%,0,0,94%


### Report

In [162]:
from reportlab.pdfgen.canvas import Canvas
from reportlab.lib.styles import ParagraphStyle as PS
from reportlab.platypus import PageBreak, Table, NextPageTemplate, Image, Spacer, Flowable, TableStyle
from reportlab.platypus.paragraph import Paragraph
from reportlab.platypus.doctemplate import PageTemplate, BaseDocTemplate
from reportlab.pdfbase.pdfmetrics import registerFont, registerFontFamily
from reportlab.platypus.tableofcontents import TableOfContents
from reportlab.platypus.frames import Frame
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib.units import cm, inch
from reportlab.lib import colors, utils
from functools import partial
from reportlab.lib.pagesizes import letter
from PIL import Image as pImage

In [274]:
from docxtpl import DocxTemplate, InlineImage
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.offsetbox import (TextArea, DrawingArea, OffsetImage, AnnotationBbox)
import jinja2

In [277]:
# Jinja function to assign variables

def create_report(data):
    i = 0
    global student, program, fulldate, vehicle, s_graph, snor, spoce, saoep, saovc, asfpl, l_graph, lnor, lpoce, laoep, laovc, lfpl, msed_rev_time, comment, items, pct_graph, mse_graph, slalom_graph, lnch_graph, rev_graph
    
    performance_report_tmplt = "../../../static/reports/"
    if not os.path.exists("plots") or not os.path.isdir("plots"):
        os.mkdir("plots")
    if not os.path.exists("reports") or not os.path.isdir("reports"):
        os.mkdir("reports")
    
    for index, row in data.iterrows():
        if i > len(data):
            break
        else:
            if country == 'MX':
                doc = DocxTemplate(f'{performance_report_tmplt}/esp_performance_report_tmplt.docx')
            else:
                doc = DocxTemplate(f'{performance_report_tmplt}eng_performance_report_tmplt.docx')
            
            #Define variables for template
            
            if row['fullname'] not in df_final_exercise['participant'].values:
                pass
            else:
                student = row['fullname']
            
            
            
            fullname = student
            studentId = row["studentId"]
            company = row['company']
            program = row['program']
            
            # Date language Format
            if country == 'MX':
                fulldate = row['date'].strftime("%d / %m / %Y")
            else:
                fulldate = row['date'].strftime("%B %d %Y")
            
            vehicle = row['vehicle']
            snor = row['s_no_runs']
            spoce = int(row['prcnt_s_pass'])
            saoep = int(row['avg_ex_control_s'])
            saovc = int(row['avg_v_control_s'])
            sfpl = int(row['slalom_max'])
            lnor = row['lc_no_runs']
            lpoce = int(row['prcnt_lc_pass'])
            laoep = int(row['avg_ex_control_lc'])
            laovc = int(row['avg_v_control_lc']) 
            lfpl = int(row['lnch_max'])
            
            #Graphs Variables
            ax1_slalom_plt = (dfs['skill_building_line'].loc[
                ((dfs['skill_building_line']['studentId'] == studentId) & 
                 (dfs['skill_building_line']['exercise'] == 'Slalom')), '%_of_exercise'
            ].astype(float))*100
            ax1_slalom_plt = ax1_slalom_plt.reset_index(drop=True)
            ax2_slalom_plt = (dfs['skill_building_line'].loc[
                ((dfs['skill_building_line']['studentId'] == studentId) & 
                 (dfs['skill_building_line']['exercise'] == 'Slalom')), '%_of_vehicle'
            ].astype(float))*100
            ax2_slalom_plt = ax2_slalom_plt.reset_index(drop=True)
            
            ax1_LnCh_plt = (dfs['skill_building_line'].loc[
                ((dfs['skill_building_line']['studentId'] == studentId) & 
                 (dfs['skill_building_line']['exercise'] == 'Lane Change')), '%_of_exercise'
            ].astype(float))*100
            ax1_LnCh_plt = ax1_LnCh_plt.reset_index(drop=True)
            ax2_LnCh_plt = (dfs['skill_building_line'].loc[
                ((dfs['skill_building_line']['studentId'] == studentId) & 
                 (dfs['skill_building_line']['exercise'] == 'Lane Change')), '%_of_vehicle'
            ].astype(float))*100
            ax2_LnCh_plt = ax2_LnCh_plt.reset_index(drop=True)
            
            # Instructor comments 
            paragraph = row['comments']
            
            #Slalom Runs Graph
            plt.style.use('seaborn-dark-palette')
            plt.figure(figsize=(6.5,2))
            plt.axhline(y=80, color='#C87867', ls='--', lw=3)
            plt.annotate('Ideal', (0, 90), ha='center', va='center', fontsize=8, color='#C87867')
            plt.axhline(y=100, color='#67BEC8', ls='--', lw=3)
            plt.annotate('Ideal', (0, 110), ha='center', va='center', fontsize=8, color='#67BEC8')
            
            
            # Language Specific
            if country == 'MX':
                plt.plot(ax1_slalom_plt, label = '% Del Ejercicio', linewidth=2, color='#001EBA')
                plt.plot(ax2_slalom_plt, label = '% Del Vehículo', linewidth=2, color='#BA0000')
                plt.title('Resultados de Slalom - ' + student)
            else:
                plt.plot(ax1_slalom_plt, label = '% Of The Exercise', linewidth=2, color='#001EBA')
                plt.plot(ax2_slalom_plt, label = '% Of The Vehicle', linewidth=2, color='#BA0000')
                plt.title(student + ' ' + 'Slalom Results')
            
            plt.ylim(ymin=0, ymax=120)
            plt.legend()
            
            
              #Language specific labels
            if country == 'MX':
                plt.xlabel('Intentos')
            else:
                plt.xlabel('Runs')
            
            plt.ylabel('%')
            plt.tight_layout()
            plt.savefig('plots/Slalom Graph ' + student + '.png', bbox_inches='tight', dpi=300)
            plt.close()
            
            s_graph = InlineImage(doc, 'plots/Slalom Graph ' + student + '.png')
            
            #Lane Change Graph
            plt.style.use('seaborn-dark-palette')
            plt.figure(figsize=(6.5,2))
            plt.axhline(y=80, color='#C87867', ls='--', lw=3)
            plt.annotate('Ideal', (0, 90), ha='center', va='center', fontsize=8, color='#C87867')
            plt.axhline(y=100, color='#67BEC8', ls='--', lw=3)
            plt.annotate('Ideal', (0, 110), ha='center', va='center', fontsize=8, color='#67BEC8')
            
               # Language Specific
            if country == 'MX':
                plt.plot(ax1_LnCh_plt, color='#001EBA', label = '% Del Ejercicio', linewidth=2)
                plt.plot(ax2_LnCh_plt, color='#BA0000', label = '% Del Vehículo', linewidth=2)
                plt.title('Resultados de Evasión de Barricada - ' + student)
            else:
                plt.plot(ax1_LnCh_plt, color='#001EBA', label = '% Of The Exercise', linewidth=2)
                plt.plot(ax2_LnCh_plt, color='#BA0000', label = '% Of The Vehicle', linewidth=2)
                plt.title(student + ' ' + 'Barricade Evasion Results')
            
            plt.ylim(ymin=0, ymax=120)
            plt.legend()
            
            #Language specific labels
            if country == 'MX':
                plt.xlabel('Intentos')
            else:
                plt.xlabel('Runs')
            
            plt.ylabel('%')
            plt.tight_layout()
            plt.savefig('plots/LnCh Graph ' + student + '.png', bbox_inches='tight', dpi=300)
            plt.close()
            
            l_graph = InlineImage(doc, 'plots/LnCh Graph ' + student + '.png')
            
            #Final exercise table population
            items = mse_report[student]
            
# Final Exercise Percetage Graph - NEW
            column_x = df_final_exercise.loc[(df_final_exercise['studentId'] == studentId)]['final_result']
            row_x = column_x.idxmax()

            # Data Variables
            data_slalom = df_final_exercise.iloc[row_x]['slalom']
            data_lnch = df_final_exercise.iloc[row_x]['LnCh']
            data_rev = df_final_exercise.iloc[row_x]['rev_%']*100
            data_s = df_final_exercise.iloc[row_x]['final_result']*100
            min_std = 80 - data_s
            if min_std < 0:
                min_std = 0
            else:
                pass
            data_y = 100 - (min_std + data_s)
            size = [data_s, min_std, data_y]
            marker_a = [79,1,20]

            dif_pct = 100 - mseg_perf
            gp_pct = [mseg_perf, dif_pct]
            # min_std = 80
            min_std_df = 100 - min_std
            avg_size = [min_std, min_std_df]
            my_circle = plt.Circle((0,0), 0.7, color='white')
            
            # Colors for percetage graph
            if data_s >= 80:
                colors_x = ['lawngreen', 'gainsboro', 'grey']
            elif data_s >= 70 < 80:
                colors_x = ['gold', 'gainsboro', 'grey']
            elif data_s < 70:
                colors_x = ['darkred', 'gainsboro', 'grey']

            # Overall Perf Graph
            fig, ax = plt.subplots(figsize=(1.5,1.5))
            ax.axis('equal')
            
            ## Local Levels
            if country == 'MX':
                label1 = """Min.80%"""
                label2 = "Desempeño"
            else:
                label1 = """Min.80%"""
                label2 = "Performance"

            pie_3_names = ['',label1, '']

            pie_back, _ = ax.pie(marker_a, radius = 1.3+.1, colors=['white', 'black', 'white'], labels=pie_3_names, counterclock = False, startangle=-90, textprops={'fontsize': 10})
            plt.setp(pie_back, width=0.3, edgecolor='white')
            pie_1, _ = ax.pie(size, radius = 1.3, colors=colors_x, counterclock = False, startangle=-90, textprops={'fontsize': 8})
            plt.setp(pie_1, width=0.4, edgecolor='white')
            ax.annotate((str(int(data_s)) + '%'), (0, 0), ha='center', va='center', fontsize=16, fontweight='bold', color='black')
#             ax.annotate((label2), (0, .35), ha='center', va='center', fontsize=8, color='black')
            plt.margins(0,0)
            plt.savefig('plots/final_percent-' + student + '.png', bbox_inches='tight', dpi=300)
#             plt.show()
            plt.close()
            
            pct_graph = InlineImage(doc, 'plots/final_percent-' + student + '.png')
            
#Slalom Gpraph
            min_std = 80 - data_slalom
            if min_std < 0:
                min_std = 0
            else:
                pass
            data_y_slalom = 100 - (min_std + data_slalom)
            size = [data_slalom, min_std, data_y_slalom]
            marker_a = [79,1,20]

            dif_pct = 100 - mseg_perf
            gp_pct = [mseg_perf, dif_pct]
            # min_std = 80
            min_std_df = 100 - min_std
            avg_size = [min_std, min_std_df]
            my_circle = plt.Circle((0,0), 0.7, color='white')

            fig, ax = plt.subplots(figsize=(1,1))
            ax.axis('equal')

            if data_slalom >= 80:
                colors_x = ['lawngreen', 'gainsboro', 'grey']
            elif data_slalom >= 70 < 80:
                colors_x = ['gold', 'gainsboro', 'grey']
            elif data_slalom < 70:
                colors_x = ['darkred', 'gainsboro', 'grey']

            ## Local Levels
            if country == 'MX':
                label1 = """Min. 80%"""
                label3 = ""
            else:
                label1 = """Min. 80%"""
                label3 = ""

            pie_3_names = ['',label1, '']

            pie_back, _ = ax.pie(marker_a, 
                                 radius = 1.3+.1, 
                                 colors=['white', 'black', 'white'], 
                                 labels=pie_3_names, 
                                 counterclock = False, 
                                 startangle=-90,
                                 textprops={'fontsize': 8},
                                 rotatelabels = False,
                                 labeldistance = 1
                                )
            plt.setp(pie_back, width=0.3, edgecolor='white')
            pie_1, _ = ax.pie(size, radius = 1.3, colors=colors_x, counterclock = False, startangle=-90, textprops={'fontsize': 6})
            plt.setp(pie_1, width=0.5, edgecolor='white')
            ax.annotate((str(int(data_slalom)) + '%'), (0, 0), ha='center', va='center', fontsize=10, fontweight='bold', color='black')
            ax.annotate((label3), (0, .35), ha='center', va='center', fontsize=6, color='black')
            plt.margins(0,0)

            plt.savefig('plots/final_slalom_percent-' + student + '.png', bbox_inches='tight', dpi=300)
            plt.close() 
            
            slalom_graph = InlineImage(doc, 'plots/final_slalom_percent-' + student + '.png') #Slalom Graph
            
#LaneChange Gpraph
            min_std = 80 - data_lnch
            if min_std < 0:
                min_std = 0
            else:
                pass
            data_y_lnch = 100 - (min_std + data_lnch)
            size = [data_lnch, min_std, data_y_lnch]
            marker_a = [79,1,20]

            dif_pct = 100 - mseg_perf
            gp_pct = [mseg_perf, dif_pct]
            # min_std = 80
            min_std_df = 100 - min_std
            avg_size = [min_std, min_std_df]
            my_circle = plt.Circle((0,0), 0.7, color='white')

            fig, ax = plt.subplots(figsize=(1,1))
            ax.axis('equal')

            if data_lnch >= 80:
                colors_x = ['lawngreen', 'gainsboro', 'grey']
            elif data_lnch >= 70 < 80:
                colors_x = ['gold', 'gainsboro', 'grey']
            elif data_lnch < 70:
                colors_x = ['darkred', 'gainsboro', 'grey']

            ## Local Levels
            if country == 'MX':
                label1 = """Min. 80%"""
                label4 = ""
            else:
                label1 = """Min. 80%"""
                label4 = ""

            pie_3_names = ['',label1, '']

            pie_back, _ = ax.pie(marker_a, 
                                 radius = 1.3+.1, 
                                 colors=['white', 'black', 'white'], 
                                 labels=pie_3_names, 
                                 counterclock = False, 
                                 startangle=-90,
                                 textprops={'fontsize': 8},
                                 rotatelabels = False,
                                 labeldistance = 1
                                )
            plt.setp(pie_back, width=0.3, edgecolor='white')
            pie_1, _ = ax.pie(size, radius = 1.3, colors=colors_x, counterclock = False, startangle=-90, textprops={'fontsize': 6})
            plt.setp(pie_1, width=0.5, edgecolor='white')
            ax.annotate((str(int(data_lnch)) + '%'), (0, 0), ha='center', va='center', fontsize=10, fontweight='bold', color='black')
            ax.annotate((label4), (0, .35), ha='center', va='center', fontsize=6, color='black')
            plt.margins(0,0)

            plt.savefig('plots/final_lnch_percent-' + student + '.png', bbox_inches='tight', dpi=300)
            plt.close()
            
            lnch_graph = InlineImage(doc, 'plots/final_lnch_percent-' + student + '.png') #LaneChange Gpraph
            
#Reverse Gpraph
            min_std = 20 - data_rev
            if min_std < 0:
                min_std = 0
            else:
                pass
            data_y_rv = 100 - (min_std + data_rev)
            size = [data_rev, min_std, data_y_rv]
            marker_a = [19,1,80]

            dif_pct = 100 - mseg_perf
            gp_pct = [mseg_perf, dif_pct]
            # min_std = 80
            min_std_df = 100 - min_std
            avg_size = [min_std, min_std_df]
            my_circle = plt.Circle((0,0), 0.7, color='white')

            fig, ax = plt.subplots(figsize=(1,1))
            ax.axis('equal')

            if data_rev > 25:
                colors_x = ['darkred', 'gainsboro', 'grey']
            elif data_rev > 20 <= 25:
                colors_x = ['gold', 'gainsboro', 'grey']
            elif data_rev <= 20:
                colors_x = ['lawngreen', 'gainsboro', 'grey']

            ## Local Levels
            if country == 'MX':
                label1 = """Max. 20%"""
                label5 = ""
            else:
                label1 = """Max. 20%"""
                label5 = ""

            pie_3_names = ['',label1, '']

            pie_back, _ = ax.pie(marker_a, 
                                 radius = 1.3+.1, 
                                 colors=['white', 'black', 'white'], 
                                 labels=pie_3_names, 
                                 counterclock = False, 
                                 startangle=-90, 
                                 textprops={'fontsize': 8},
                                 rotatelabels = False,
                                 labeldistance = 1
                                )
#             print(student, size)
            plt.setp(pie_back, width=0.3, edgecolor='white')
            pie_1, _ = ax.pie(size, radius = 1.3, colors=colors_x, counterclock = False, startangle=-90, textprops={'fontsize': 6})
            plt.setp(pie_1, width=0.5, edgecolor='white')
            ax.annotate((str(int(data_rev)) + '%'), (0, 0), ha='center', va='center', fontsize=10, fontweight='bold', color='black')
            ax.annotate((label5), (0, .35), ha='center', va='center', fontsize=6, color='black')
            plt.margins(0,0)

            plt.savefig('plots/final_rv_percent-' + student + '.png', bbox_inches='tight', dpi=300)
            plt.close()
            
            rev_graph = InlineImage(doc, 'plots/final_rv_percent-' + student + '.png') #Reverse Gpraph
            
# Final Exercise Graph -New


            data_time = df_final_exercise.loc[df_final_exercise['studentId'] == studentId].f_time.dt.total_seconds()
            data_time = data_time.reset_index(drop=True)
            data_performance = df_final_exercise.loc[df_final_exercise['studentId'] == studentId].final_result*100
            data_performance = data_performance.reset_index(drop=True)
            data_rv = df_final_exercise.loc[df_final_exercise['studentId'] == studentId].rev_slalom.dt.total_seconds()
            data_rv = data_rv.reset_index(drop=True)
            
            
            data_tm_fn = data_performance
            cones_fn = df_final_exercise.loc[df_final_exercise['studentId'] == studentId].cones
            cones_fn = cones_fn.reset_index(drop=True)
            gates_fn = df_final_exercise.loc[df_final_exercise['studentId'] == studentId].gates
            gates_fn = gates_fn.reset_index(drop=True)
            xtick = mse_report[student][0]['f_time']
            assets_dir = "../../../static/assets"
            cone_img = mpimg.imread(f'{assets_dir}/cones_for_reports.png')

            av_rev_result = df_final_exercise['rev_slalom'].mean().total_seconds()
            av_gp_time = df_final_exercise['f_time'].mean().total_seconds()

            pass_time = 80 #(ideal_time.total_seconds()*1.2)
            
            # New Values for Graphs #<-----Changed
            data_rv_pct = (data_rv / data_time)*100
            data_per_fn = data_performance - data_rv_pct

            slalom_pct = df_final_exercise.loc[df_final_exercise['studentId'] == studentId].slalom
            slalom_pct = slalom_pct.reset_index(drop=True)
            lnch_pct = df_final_exercise.loc[df_final_exercise['studentId'] == studentId].LnCh
            lnch_pct = lnch_pct.reset_index(drop=True)

            passes = data_time.index.tolist()
            passes.append(2)

            label = df_final_exercise.loc[df_final_exercise['studentId'] == studentId].stress
            if country == 'MX':
                label = label.reset_index(drop=True).replace(0, "Bajo Estrés").replace(1, 'Alto Estrés')
            else:
                label = label.reset_index(drop=True).replace(0, "Low Stress").replace(1, 'High Stress')


            data_sx = mse_report[student][0]['final_result']
            min_stdx = 80 - data_sx
            if min_stdx < 0:
                min_stdx = 0
            else:
                pass
            data_yx = 100 - (min_std + data_s)

            barHeight = .8

            fig, ax = plt.subplots(figsize=(8,4))
            
            ax.invert_yaxis()
            
            imagebox = OffsetImage(cone_img, zoom=0.1)
            imagebox.image.axes = ax

            # Average Bar
            ## labels
            if country == 'MX':
                label3 = """Promedio: """
                label4 = 'Prom. Rev: '
                label5 = """ Prom. Penalización: """
                label6 = 'Prom. Gpo. '
            else:
                label3 = """Average: """
                label4 = 'Av. Rev: '
                label5 = """ Av. Penalties: """
                label6 = 'Gp. Average '
            
            ax.barh('Av', float(mseg_rev), color='khaki', edgecolor='white', height=.3) #<-----Changed
            ax.barh('Av', float(mseg_perf)-float(mseg_rev), left=float(mseg_rev), color='gold', edgecolor='white', height=.3)#<-----Changed
            
            ax.annotate(label3 + str(mseg_t), (mseg_perf, .3), color='black', va='center', ha='right', fontsize=8)
            ax.annotate(label4 + str(mseg_rev), (2, .3), color='black', va='center', ha='left', fontsize=8)
#             ax.annotate(label5 + str(mseg_c) + " | " + str(mseg_g), (mseg_perf, .3), color='black', fontsize=8, va='center', ha='center')
            ax.annotate(label5 + str(mseg_c) + " | " + str(mseg_g), (float(mseg_rev) + 10, .3), color='black', fontsize=8, va='center', ha='left')
            ax.annotate(label6 + str(mseg_perf) + '%', xy=(float(mseg_perf) + .3, 0), xytext=(float(mseg_perf) + 10, 0), ha='left', fontsize=10,
                       arrowprops=dict(arrowstyle='->'), va='center')

            # Main Bar Plot
            ## Labels ##
            if country == 'MX':
                label7 = """T: """ 
                label8 = """Aceptable"""
                label9 = """Penalizaciones: """
            else:
                label7 = """T: """ 
                label8 = """Acceptable"""
                label9 = """Penalties: """

            for i, v in data_tm_fn.items():
            
            # Student Bar
                ax.barh(i+1, data_rv_pct[i], 
                        color='darkred', 
                        edgecolor='white', 
                        height=barHeight) 
                ax.barh(i+1, v - data_rv_pct[i], 
                        left=data_rv_pct[i], 
                        color='red', 
                        edgecolor='white', 
                        height=barHeight) 

                xtick1 = (mse_report[student][i]['f_time'])
                xtick2 = xtick1 #str(datetime.strptime(xtick1, '%M:%S.%f').time()) <------Deprecated (No se necesita con el cambio anterior)
                
                # Time Annotations
                ax.annotate(label7 + xtick2, 
                            (v - 1, i+1), 
                            color='w', fontsize=10, 
                            fontweight='bold', 
                            va='center', 
                            ha='right')
                ax.annotate(str(int(data_performance[i]))+'%', 
                            (v + 1, i+1), 
                            color='black', 
                            fontsize=10, 
                            fontweight='bold', 
                            va='center', 
                            ha='left')

            # Acceptable Reverse Annotation
#                 ax.annotate('', (.1, i + 1.4), xytext=((80 * .2), i + 1.4), #<------ Hay un error en esta formula, no marca la realidad
#                             arrowprops=dict(arrowstyle='|-|'), 
#                             va='top')
#                 ax.annotate(label8,(1, i + 1.5), 
#                             xytext=(80 * .10, i + 1.5), 
#                             va='center', ha='center', fontsize=8)

            # Stress Labels
                if label[i] == "High Stress" or "Alto Estrés":
                    label_color = 'orangered'
                else:
                    label_color = 'yellowgreen'
                
                ax.annotate(label[i], 
                            ((v)+1, i + .8), 
                            ha='left', va='top', 
                            fontsize=8, fontweight='bold', 
                            color=(label_color))
                
                
                # Add Penalties Image
                ab = AnnotationBbox(imagebox, 
                                    ((data_performance[i] - data_rv_pct[i]) - 11, i+1.4),     # Bar Image 
                                    xybox=(0., 0.), 
                                    xycoords='data',
                                    boxcoords="offset points", 
                                    frameon=False)
                ax.add_artist(ab)
                
                # Penalties Cones | Gates
                ax.annotate(label9 + str(cones_fn[i]) + " | " + str(gates_fn[i]), 
                            (data_performance[i] - data_rv_pct[i], i+1.5), 
                            color='black', 
                            fontsize=8, 
                            ha='center', va='center')
                
                # Slalom | LnCh Percetage Annotation
                if data_performance[i] < 60:
                    label10 = 'S' 
                    label11 = 'L'
                    labels_offset = .8
                else:
                    label10 = 'Slalom'
                    label11 = 'LnCh'
                    labels_offset = 1
                ax.annotate(label10 + ' ' + str(slalom_pct[i]) + '%' + " | " + label11 + ' ' + str(lnch_pct[i]) + '%', 
                            (data_rv_pct[i] + 5, i + labels_offset), 
                            color='w', 
                            fontsize = 10, fontweight='bold', 
                            va='center', ha='left')

            
            # Reverse Time Annotation
            for i, v in data_rv.items():
                if country == 'MX':
                    ax.annotate("Reversa:" + str(round(float(v),2)), (2, i+1), color='w', fontsize=10, fontweight='bold', va='center') #Reverse Bar Legend
                else:
                    ax.annotate("Reverse:" + str(round(float(v),2)), (2, i+1), color='w', fontsize=10, fontweight='bold', va='center') #Reverse Bar Legend

            # Pass or Fail Annotation <-----Changed
            ax.axvline(pass_time, linestyle='--', ymax=len(passes), color='aqua')
            if country == 'MX':
                ax.annotate('Necesita Práctica', xy=(pass_time - 23, len(passes)-.3), xytext=(pass_time -1, len(passes)-.3), ha='right', fontsize=10,
                       arrowprops=dict(arrowstyle='->'), va='center')
                ax.annotate('Security Driver', xy=(pass_time + 20, len(passes)-.3), xytext=(pass_time +1, len(passes)-.3), fontsize=10,
                       arrowprops=dict(arrowstyle='->'), va='center')
            else:
                ax.annotate('Needs More Work', xy=(pass_time - 30, len(passes)-.3), xytext=(pass_time -1, len(passes)-.3), ha='right', fontsize=10,
                       arrowprops=dict(arrowstyle='->'), va='center')
                ax.annotate('Security Driver', xy=(pass_time + 20, len(passes)-.3), xytext=(pass_time +1, len(passes)-.3), fontsize=10,
                       arrowprops=dict(arrowstyle='->'), va='center')

            ax.set_yticks([])
            ax.set_xticks([])
            ax.axis('off')

            fig.tight_layout()
            plt.xlim(xmin=0, xmax=100)
            plt.ylim(ymin=-.5, ymax=len(passes))
            plt.savefig('plots/final_exercise-' + student + '.png', bbox_inches='tight', dpi=300)
#             plt.show()
            plt.close()
            
            mse_graph = InlineImage(doc, 'plots/final_exercise-' + student + '.png')
            
            for var in tmplt_context:
                tmplt_context[var] = eval(var)
            
            jinja_env = jinja2.Environment(autoescape=True)
            doc.render(tmplt_context, jinja_env)
            
            doc.save('reports/' + str(student) + ' - Report.docx')
            
            comment = str(paragraph)
            
            i+=1
            
            

In [278]:
create_report(df_report)

In [279]:
# Variables
company = dfs["course_generals"].loc[0, "client"]
fulldate = dfs["course_generals"].loc[0, "date"]
vehicle = dfs["car_information"].loc[0, "make"]

page_size = letter
pwidth, pheight = letter
fwidth = pwidth*.8

'-----------------------------------------------------------------------------'
'Register Fonts'
'-----------------------------------------------------------------------------'

font_dir = '../../../static/reports/fonts'
registerFont(TTFont('MontserratBold', f"{font_dir}/Montserrat ExtraBold 800.ttf"))
registerFont(TTFont('MontserratBlack', f'{font_dir}/Montserrat Black 900.ttf'))
registerFont(TTFont('Montserrat', f'{font_dir}/Montserrat-Regular.ttf'))
registerFont(TTFont('MontserratLight', f'{font_dir}/Montserrat Light 300.ttf'))
registerFont(TTFont('MontserratThin', f'{font_dir}/Montserrat Thin 250.ttf'))

from reportlab.lib.fonts import addMapping
# addMapping('SabonRom', 0, 0, 'SabonRom') #normal
# addMapping('SabonRom', 0, 1, 'SabonIta') #italic
addMapping('MontserratLight', 1, 0, 'MontserratBold') #bold
# addMapping('SabonRom', 1, 1, 'SabonBolIta') #italic and bold
# heavy = ParagraphStyle(name='normal', fontName='SabonRom', fontSize=10, leading=1.4*10 )


'-----------------------------------------------------------------------------'
'Template'
'-----------------------------------------------------------------------------'

class MyDocTemplate(BaseDocTemplate):

    def __init__(self, filename, **kw):
        self.allowSplitting = 0
        BaseDocTemplate.__init__(self, filename, **kw)

        template_CoverPage = PageTemplate('CoverPage',
            [Frame(.8*cm, pheight/2-(inch*3), pwidth/2, 230, id='F1', showBoundary=0)], 
            onPage=createCover, 
            )
        
        template_NormalPage = PageTemplate('NormalPage',
                                [Frame(pwidth*.08, 2.5*cm, pwidth*.84, pheight*.8, id='F2', 
                                    showBoundary=0)],
                                onPage=partial(header_and_footer,
                                header_content=header_content,
                                footer_content=footer_content),
                                pagesize=page_size)
        
        template_SecondPage = PageTemplate('SecondPage',
                                [Frame(pwidth*.08, pheight*.37, pwidth*.84, pheight*.50, id='F3', 
                                    showBoundary=0),
                                Frame(pwidth*.08, 3*cm, pwidth*.42, pheight*.25, id='F4', 
                                    showBoundary=0),
                                Frame(pwidth*.50, 3*cm, pwidth*.42, pheight*.25, id='F5', 
                                    showBoundary=0)],
                                onPage=partial(header_and_footer,
                                header_content=header_content,
                                footer_content=footer_content),
                                pagesize=page_size)

        self.addPageTemplates([template_CoverPage,template_NormalPage, template_SecondPage])



    def afterFlowable(self, flowable):
        "Registers TOC entries."
        if flowable.__class__.__name__ == 'Paragraph':
            text = flowable.getPlainText()
            style = flowable.style.name
            if style == 'Heading1':
                self.notify('TOCEntry', (0, text, self.page))
            if style == 'Heading2':
                self.notify('TOCEntry', (1, text, self.page))

h1 = PS(name = 'Heading1',
       fontSize = 18,
       leading = 20,
       fontName='MontserratBold',
       textColor="#C10230",
       spaceAfter = 8,
       spaceBefore = 8)

h2 = PS(name = 'Heading2',
       fontSize = 14,
       fontName='MontserratBlack',
       leading = 16,
       leftIndent = 0)
h3 = PS(name = 'Heading3',
       fontSize = 14,
       fontName='MontserratBold',
       textColor="#C10230",
       leading = 16,
       leftIndent = 10)
subh2 = PS(name = 'SubHeading2',
       fontSize = 14,
       fontName='Montserrat',
       leading = 16,
       leftIndent = 5)
l0 = PS(name = 'list0',
        fontSize = 12,
        leading =14,
        leftIndent=0,
        rightIndent=0,
        spaceBefore = 12,
        spaceAfter =0
        )
pstyle = PS(name='ms',
        fontName='MontserratLight',
        fontSize=10,
        leading =14,
        spaceAfter = 8,
        spaceBefore = 8
        )
no_spacing = PS(name='ms',
        fontName='MontserratLight',
        fontSize=10,
        leading =14,
        spaceAfter = 0,
        spaceBefore = 0
        )
footertxt = PS(name='footer',
        fontName='MontserratLight',
        fontSize=10,
        leading =9,
        leftIndent=40,
        rightIndent=0,
        )
super_h1 = PS(name = 'Super_Heading1',
       fontSize = 80,
       leading = 0,
       fontName='MontserratBold',
       textColor="#C10230",
       spaceAfter = 0,
       spaceBefore = 0)
ttl = PS(name = 'Title',
       fontSize = 24,
       leading = 25,
       fontName='MontserratBlack',
       textColor="#000000",
       spaceAfter = 0,
       spaceBefore = 0)

'-----------------------------------------------------------------------------'
'Table Styles'
'-----------------------------------------------------------------------------'

normalTable = TableStyle([
    ('FONTNAME', (0,0), (0, -1), 'Montserrat'),
    ('FONTNAME', (0,-1), (-1,-1), 'Montserrat'),
    ('FONTNAME', (0,1), (-1,-2), 'MontserratLight'),
#     ('GRID',(0,0),(-1,-1),.5,colors.black),
    ('ALIGN', (0,0), (-1,0), 'CENTER'),
    ('ALIGN', (1,1), (-1,-1), 'CENTER'),
    ('LINEBELOW', (0,0), (-1,0), 2, colors.black),
    ('LINEABOVE', (0,-1), (-1,-1), .5, colors.black),
    ('LINEBELOW', (0,-1), (-1,-1), 2, colors.black),
])

mseTable = TableStyle([
    ('FONTNAME', (0,0), (0, -1), 'Montserrat'),
    ('FONTNAME', (0,-1), (-1,-1), 'Montserrat'),
    ('FONTNAME', (0,1), (-1,-2), 'MontserratLight'),
#     ('GRID',(0,0),(-1,-1),.5,colors.black),
    ('ALIGN', (1,0), (-1,0), 'CENTER'),
    ('ALIGN', (1,1), (-1,-1), 'CENTER'),
    ('LINEBELOW', (0,0), (-1,0), 2, colors.black),
    ('LINEABOVE', (0,-1), (-1,-1), .5, colors.black),
    ('LINEBELOW', (0,-1), (-1,-1), 2, colors.black),
    ('BACKGROUND', (0, -1), (-1,-1), colors.lightcoral),
    ('VALIGN', (0, -1), (-1,-1), 'MIDDLE'),
    
])

introTable = TableStyle([
    ('FONTNAME', (0,0), (-1, -1), 'Montserrat'),
#     ('GRID',(0,0),(-1,-1),.5,colors.black),
    ('ALIGN', (0,0), (-1,-1), 'LEFT'),
    ('LINEBELOW', (0,0), (-1,-1), 1, colors.black),
])

consoleTable = TableStyle([
    ('BOX',(0,0),(-1,-1), 2, colors.firebrick),
    ('ALIGN', (0,0), (0,-1), 'RIGHT'),
    ('ALIGN', (0,1), (-1,-1), 'CENTER'),
    ('VALIGN', (0,1), (-1,-1), 'TOP'),
    ('BACKGROUND', (0, 0), (-1,-1), colors.beige),
#     ('LINEBELOW', (0,0), (-1,-1), 1, colors.black),
])


subTable = TableStyle([
    ('FONTNAME', (0,0), (-1,-1), 'MontserratLight'),
#     ('GRID',(0,0),(-1,-1),.5,colors.black),
    ('VALIGN', (0,0), (-1,-1), 'TOP'),
    ('ALIGN', (0,0), (-1,-1), 'RIGHT'),
    ('TOPPADDING', (0,0), (0,0), 10)
])

'-----------------------------------------------------------------------------'
'Header and Footer'
'-----------------------------------------------------------------------------'
def header(canvas, doc, content):
    canvas.saveState()
    w, h = content.wrap(doc.width, doc.topMargin)
    content.drawOn(canvas, doc.leftMargin-22.4, doc.height + doc.bottomMargin + doc.topMargin - h -10)
    canvas.restoreState()

def footer(canvas, doc, content):
    drawPageNumber(canvas, doc)
    canvas.saveState()
    w, h = content.wrap(doc.width, doc.bottomMargin)
    content.drawOn(canvas, doc.leftMargin-22.4, h)
    canvas.restoreState()

def header_and_footer(canvas, doc, header_content, footer_content):
    header(canvas, doc, header_content)
    footer(canvas, doc, footer_content)

def drawPageNumber(canvas, doc):
    pageNumber = canvas.getPageNumber()
    # canvas.setFont("Helvetica",11)
    # canvas.drawCentredString(17.4*cm, 1.35*cm, 'Page '+str(pageNumber))

def PageNumber(canvas, doc):
    return(canvas.getPageNumber())

header_center_text='header_center_text foo'
footer_center_text='footer_center_text bar' 

'-----------------------------------------------------------------------------'
'Variables and Texts'
'-----------------------------------------------------------------------------'

logo_dir = '../../../static/reports/logos'
logo_int = f'{logo_dir}/Logo---AS3-international-200x200.png'
ceo_cfo = f'{logo_dir}/The-CFO-to-the-CEO---Logo.png'
ceo_cfo_inv = f'{logo_dir}/The-CFO-to-the-CEO---Logo_inverted.png'
as3_logo = f'{logo_dir}/AS3 Driver Training - Logo.png'
chrysler300 = f'{logo_dir}/chrysler_300.png'

text_dir = '../../../static/reports/texts'
if country == 'MX':
    with open(f'{text_dir}/[es]_personal_report_intro.txt', 'r', encoding="utf-8") as file:
        intro_pre = file.read()
        intro_text = intro_pre
    
    with open(f'{text_dir}/[es]slalom_description.txt', 'r', encoding="utf-8") as file:
        slalom_pre = file.read()
        slalom_text = slalom_pre

    with open(f'{text_dir}/[es]final_x_description.txt', 'r', encoding="utf-8") as file:
        mse_pre = file.read()
        mse_text = mse_pre
        
    with open(f'{text_dir}/[es]final_x_intro.txt', 'r', encoding="utf-8") as file:
        mse_pre2 = file.read()
        mse_desc = mse_pre2
    
    with open(f'{text_dir}/[es]final_note.txt', 'r', encoding="utf-8") as file:
        fn_note_pre = file.read()
        fn_note = fn_note_pre
    
    
    mainTitle = 'EVALUACIÓN POST ENTRENAMIENTO (AAR)'
    scndTitle = '<para alignment="center">EVALUACIÓN DE DESEMPEÑO</para>'
    thirdTitle = 'DESEMPEÑO EN EJERCICIOS'
    thirdSubTitle = 'EJERCICIOS DE DESARROLLO DE HABILIDADES'
    fourthSubTitle = 'EJERCICIO FINAL CONBINADO'
    slalomTitle = '<para><b>SLALOM</b> - 4 Conos, Cuerda 13mts</para>'
    LnChTitle = '<para><b>CAMBIO EVASIVO DE CARRIL</b> - Diseño Estándar, Cuerda de 32 mts</para>'
    mse_title = 'EJERCICIO MULTIDISCIPLINARIO – 9 Obstáculos, emboscada fija / en movimiento.'
    mse_consoleTitle = "KPI Personal - Mejor Resultado"
    inst_comments_title = 'COMENTARIOS DEL INSTRUCTOR LIDER'
    stdt_name = '<para><b><i>Nombre del Estudiante</i></b></para>'
    company_label = '<para><b><i>Empresa</i></b></para>'
    pgm_label = '<para><b><i>Programa</i></b></para>'
    pgmDt_label = '<para><b><i>Fecha del Programa</i></b></para>'
    vhcl_label = '<para><b><i>Vehículo</i></b></para>'
    intro_title = 'DESCRIPCIÓN DEL PROGRAMA'
    perf_table_title = 'DESEMPEÑO EN NÚMEROS'
    perf_table_heading = 'Promedio del Grupo'
    perf_table_runs = 'Número de Intentos'
    perf_table_completed = 'Porcentaje de Ejercicios Completados'
    perf_table_percetage = 'Promedio de Porcentaje del Ejercicio'
    perf_table_control = 'Promedio de Control del Vehículo'
    perf_table_maxControl = 'Nivel Máximo de Control Alcanzado'
    mse_barricade = '<para><font fontname="MontserratBold" color="darkred" size="14">Barricada</font></para>'
    mse_slalom = '<para><font fontname="MontserratBold" color="darkred" size="14">Slalom</font></para>'
    mse_reverse = '<para><font fontname="MontserratBold" color="darkred" size="14">Reversa</font></para>'
    mse_ova = '<para><font fontname="MontserratBold" color="darkred" size="14">Desempeño Global</font></para>'
    mse_graphTitle = 'Gráfica de Desempeño'
    mseTable_headers = [['Estrés', 'Tiempo Final', 'Reversa', 'Slalom', 'Barricada', Paragraph('<para align="center">Conos</para>'), 
                         Paragraph('<para align="center">Puertas</para>'), Paragraph('<para align="center">Resultado<br/> Final</para>')]]
    gpavg_lvl = 'Promedio del Grupo'

else:
    with open(f'{text_dir}/[en]_personal_report_intro.txt', 'r', encoding="utf-8") as file:
        intro_pre = file.read()
        intro_text = intro_pre
    
    with open(f'{text_dir}/[en]slalom_description.txt', 'r', encoding="utf-8") as file:
        slalom_pre = file.read()
        slalom_text = slalom_pre

    with open(f'{text_dir}/[en]final_x_description.txt', 'r', encoding="utf-8") as file:
        mse_pre = file.read()
        mse_text = mse_pre
        
    with open(f'{text_dir}/[en]final_x_intro.txt', 'r', encoding="utf-8") as file:
        mse_pre2 = file.read()
        mse_desc = mse_pre2
    
    with open(f'{text_dir}/[en]final_note.txt', 'r', encoding="utf-8") as file:
        fn_note_pre = file.read()
        fn_note = fn_note_pre
    
    mainTitle = 'AFER ACTION REPORT (AAR)'
    scndTitle = '<para alignment="center">PERFORMANCE EVALUATION</para>'
    thirdTitle = 'EXERCISE PERFORMANCE'
    thirdSubTitle = 'SKILL DEVELOPMENT EXERCISES'
    fourthSubTitle = 'COMBINED FINAL EXERCISE'
    slalomTitle = '<para><b>SLALOM</b> - 4 Cones, 50ft Chord</para>'
    LnChTitle = '<para><b>EVASIVE LANE CHANGE</b> - Standard Design, 100ft Chord</para>'
    mse_title = 'MULTI-DISCIPLINARY EXERCISE – 9 Obstacles, fixed / rolling ambush.'
    mse_consoleTitle = "Personal KPI - Top Result"
    inst_comments_title = 'LEAD INSTRUCTOR FEEDBACK'
    stdt_name = '<para><b><i>Student Name</i></b></para>'
    company_label = '<para><b><i>Company</i></b></para>'
    pgm_label = '<para><b><i>Program</i></b></para>'
    pgmDt_label = '<para><b><i>Program Date</i></b></para>'
    vhcl_label = '<para><b><i>Vehicle</i></b></para>'
    intro_title = 'PROGRAM DESCRIPTION'
    perf_table_title = 'PERFORMANCE IN NUMBERS'
    perf_table_heading = 'Group Average'
    perf_table_runs = 'Number of Runs'
    perf_table_completed = 'Percentage of Completed Exercises'
    perf_table_percetage = 'Average of Exercise Percentage'
    perf_table_control = 'Average of Vehicle Control'
    perf_table_maxControl = 'Higest Level of Control Achieved'
    mse_barricade = '<para><font fontname="MontserratBold" color="darkred" size="14">Barricade</font></para>'
    mse_slalom = '<para><font fontname="MontserratBold" color="darkred" size="14">Slalom</font></para>'
    mse_reverse = '<para><font fontname="MontserratBold" color="darkred" size="14">Reverse</font></para>'
    mse_ova = '<para><font fontname="MontserratBold" color="darkred" size="14">Overall Performance</font></para>'
    mse_graphTitle = 'Performance Graph'
    mseTable_headers = [['Stress', 'Final Time', 'Reverse', 'Slalom', 'Barricade', Paragraph('<para align="center">Cone<br/> Penalties</para>'), 
                         Paragraph('<para align="center">Gate<br/> Penalties</para>'), Paragraph('<para align="center">Final<br/> Result</para>')]]
    gpavg_lvl = 'Group Average'
    
'-----------------------------------------------------------------------------'
'Functions'
'-----------------------------------------------------------------------------'

def createCover(canvas, doc):
#     page_width, page_height = canvas._pagesize
    image = pImage.open(logo_int)
    image_width, image_height = image.size
#     draw_width, draw_height = page_width, page_height

#     canvas.drawImage(logo_int,
#                      pwidth/2, 500, #width=draw_width, height=draw_height,
#                      preserveAspectRatio=True)
    canvas.drawImage(logo_int, ((pwidth/2)-(image_width/2)),pheight-image_height*1.5, mask='auto')
    
def create_deco():
    global header_content, footer_content
    '-----------------------------------------------------------------------------'
    'Header'
    '-----------------------------------------------------------------------------'
    Header_caption=((header_center_text))
    A='Author'
    Header_table_data=[[Image(ceo_cfo_inv, width=230, height=73)]
    ]
    
    Header_table=Table(Header_table_data,
                        colWidths=[pwidth*.84],
                        rowHeights=[3*cm],
                        style=[
    # ('GRID',(0,0),(1,2),1,colors.black),
    # #('ALIGN',(0,0),(1,0),'CENTER'),
    ('ALIGN',(0,0),(0,0),'RIGHT'),
    # ('VALIGN',(0,0),(3,2),'MIDDLE'),
    # ('ALIGN',(2,0),(3,2),'RIGHT'),
    # ('SPAN',(0,0),(0,2)),
    # ('SPAN',(1,0),(1,2)),
    # ('BOX',(2,0),(-1,-1),1,colors.black),
    #('TEXTCOLOR',(0,0),(3,0),colors.Color(49/255,71/255,137/255))
    ])
    '-----------------------------------------------------------------------------'
    'Footer'
    '-----------------------------------------------------------------------------'

    Footer_table_data=[[Image(as3_logo, 
                        width=144, 
                        height=60),
                        Paragraph('<para fontname="MontserratBold" size="12" color="black">WWW.</para><para color="#C10230">AS3</para><para color="black">INTERNATIONAL.COM</para>')]
                        ]
    Footer_table=Table(Footer_table_data,colWidths=[(pwidth*.84)/2,(pwidth*.84)/2],rowHeights=[1*cm] ,style=[
    # ('GRID',(0,0),(-1,-1),1,colors.black),
    #('ALIGN',(0,0),(1,0),'CENTER'),
    ('ALIGN',(0,1),(-1,-1),'RIGHT'),
    ('VALIGN',(0,0),(0,0),'MIDDLE'),
    # ('ALIGN',(0,0),(3,0),'CENTER'),
    #('TEXTCOLOR',(0,0),(3,0),colors.Color(49/255,71/255,137/255))
    ])
    
    footer_content = Footer_table
    header_content = Header_table

create_deco() 

'Image Aspect Ratio'

def get_image(path, width=1*cm):
    img = utils.ImageReader(path)
    iw, ih = img.getSize()
    aspect = ih / float(iw)
    return Image(path, width=width, height=(width * aspect))

def altBackground(data, table):
    data_len = len(data)+1

    for each in range(data_len):
        if each % 2 == 0:
            bg_color = colors.white
        else:
            bg_color = colors.lightgrey

        table.setStyle(TableStyle([('BACKGROUND', (0, each), (-1, each), bg_color)]))

'-----------------------------------------------------------------------------'
'Build story'
'-----------------------------------------------------------------------------'

story = []

'-----------------------------------------------------------------------------'
'Intro Page'
'-----------------------------------------------------------------------------'
story.append(Paragraph(mainTitle, ttl))
story.append(Spacer(0,20))
story.append(Paragraph(student, h2))
story.append(Paragraph(company, no_spacing))
story.append(Paragraph(program, no_spacing))
story.append(NextPageTemplate('NormalPage'))
story.append(PageBreak())

'- Page 2 -'
intro_table_data = [[Paragraph(stdt_name), student],
                    [Paragraph(company_label), company],
                    [Paragraph(pgm_label), program],
                    [Paragraph(pgmDt_label), fulldate],
                    [Paragraph(vhcl_label), vehicle]
                   ]

intro_table = Table(intro_table_data, colWidths=[120,200], style=introTable)

story.append(Spacer(0,20))
story.append(Paragraph(scndTitle, ttl))
story.append(Spacer(0,20))
story.append(intro_table)
story.append(Spacer(0,20))
story.append(Paragraph(intro_title, h1))
story.append(Spacer(0,20))
story.append(Paragraph(intro_text, pstyle))
story.append(PageBreak())

'- Page 3 -'
story.append(Paragraph(thirdTitle, h1))
story.append(Paragraph(thirdSubTitle, h2))
story.append(Spacer(0,50))
story.append(Paragraph(slalomTitle, subh2))
story.append(Spacer(0,20))

slalomGraph = Image('plots/Slalom Graph ' + student + '.png', width=pwidth-inch)
# story.append(slalomGraph)
story.append(get_image('plots/Slalom Graph ' + student + '.png', width=pwidth-inch))
story.append(Paragraph(slalom_text, style=footertxt))
story.append(Spacer(0,80))
story.append(Paragraph(perf_table_title, h3))
'------------------------------------------------------------------------------------'
slalom_table_data = [['','',perf_table_heading],
                     [perf_table_completed, snor, gasnor],
                     [perf_table_runs, spoce, gaspoce],
                     [perf_table_percetage, saoep, gasaoep],
                     [perf_table_control, saovc, gasaovc],
                     [perf_table_maxControl, '', str(sfpl)+'%']
                    ]
slalom_table = Table(slalom_table_data, colWidths=[pwidth*.40, pwidth*.20, pwidth*.20], style=normalTable)
altBackground(slalom_table_data, slalom_table)
'------------------------------------------------------------------------------------'
story.append(slalom_table)
story.append(PageBreak())

'- Page 4 -'
story.append(Spacer(0,50))
story.append(Paragraph(LnChTitle, subh2))
story.append(Spacer(0,20))

slalomGraph = Image('plots/LnCh Graph ' + student + '.png', width=pwidth-inch)
# story.append(slalomGraph)
story.append(get_image('plots/LnCh Graph ' + student + '.png', width=pwidth-inch))
story.append(Paragraph(slalom_text, style=footertxt))
story.append(Spacer(0,80))
story.append(Paragraph(perf_table_title, h3))
'------------------------------------------------------------------------------------'
lnch_table_data = [['','',perf_table_heading],
                     [perf_table_completed, lnor, galnor],
                     [perf_table_runs, lpoce, galpoce],
                     [perf_table_percetage, laoep, galaoep],
                     [perf_table_control, laovc, galaovc],
                     [perf_table_maxControl, '', str(lfpl)+'%']
                    ]
lnch_table = Table(lnch_table_data, colWidths=[pwidth*.40, pwidth*.20, pwidth*.20], style=normalTable)
altBackground(lnch_table_data, lnch_table)
'------------------------------------------------------------------------------------'


story.append(lnch_table)
story.append(PageBreak())

'- Page 5 -'

story.append(Paragraph(fourthSubTitle, h1))
story.append(Spacer(0,20))
story.append(Paragraph(mse_desc))
story.append(Spacer(0,20))
story.append(Paragraph(mse_consoleTitle, h3))
story.append(Spacer(0,10))

'------------------------------------------------------------------------------------'
mse_lnch_graph = get_image('plots/final_lnch_percent-' + student + '.png', width=fwidth*.20)
mse_slalom_graph = get_image('plots/final_slalom_percent-' + student + '.png', width=fwidth*.20)
mse_reverse_graph = get_image('plots/final_rv_percent-' + student + '.png', width=fwidth*.20)
mse_ova_graph = get_image('plots/final_percent-' + student + '.png', width=fwidth*.30)


mse_table_data = [[Paragraph(mse_barricade), Paragraph(mse_slalom), Paragraph(mse_reverse), Paragraph(mse_ova)], 
                  [mse_lnch_graph, mse_slalom_graph, mse_reverse_graph, mse_ova_graph]
                 ]
mse_table = Table(mse_table_data, colWidths=[fwidth*.23, fwidth*.23, fwidth*.23, fwidth*.34], style=consoleTable)
'------------------------------------------------------------------------------------'

story.append(mse_table)
story.append(Spacer(0,5))
story.append(Paragraph(mse_text))
story.append(PageBreak())

'- Page 6 -'
msei = get_image('plots/final_exercise-' + student + '.png', width=fwidth)
msei_data = [[msei]]

# story.append(get_image('plots/final_exercise-' + student + '.png', width=fwidth))
story.append(Spacer(0,20))

'------------------------------------------------------------------------------------'
mse_hist_last_line = [[Paragraph(gpavg_lvl), mseg_t, str(mseg_rev_pc)+'%', '','', mseg_c, mseg_g, str(mseg_perf)+'%']]
mse_hist_data = mseTable_headers + np.array(mse_data.values).tolist() + np.array(mse_hist_last_line).tolist()

mse_table = Table(mse_hist_data, colWidths=[pwidth*.1, pwidth*.1, pwidth*.1, pwidth*.1, pwidth*.1, pwidth*.1, pwidth*.1, pwidth*.1], style=mseTable)

story.append(Paragraph(mse_graphTitle, h3))
story.append(Spacer(0,10))
story.append(mse_table)
story.append(Spacer(0,20))
story.append(Table(msei_data, colWidths=None, rowHeights=None, style=consoleTable))
story.append(Spacer(0,20))
story.append(Paragraph(inst_comments_title, h3))
story.append(Spacer(0,20))
story.append(Paragraph(comment))
story.append(Spacer(0,20))
story.append(Paragraph(fn_note))

'----------------------------------------------------------------'
'Document Creation'
'----------------------------------------------------------------'

doc = MyDocTemplate('AS3_Student_Report '+ company + '_Sample.pdf', pagesize=letter)
doc.multiBuild(story)

1